In [1]:
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, Slider, Button, BasicTickFormatter, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

from bokeh.plotting import figure
from bokeh.models import GraphRenderer, Ellipse, StaticLayoutProvider
from bokeh.palettes import Spectral8

import math
import numpy as np
from collections import deque
from copy import deepcopy
from pprint import pprint

output_notebook()

Loading BokehJS ...

In [2]:
class model1:
    def __init__(self):
        self.n = 1        # no. of divisions of committed slender
        self.R = 5.0      # replication time of slenders (hours)
        self.T = 24.0     # mean stumpy lifespan (hours)
        self.beta = 3e-10 # differentiation constant
        self.gamma = 0.04 # SIF degradation rate
        self.logN0 = 1.0  # number of slenders at t=0

    def simulation(self):
        """time step is 1 cell cycle"""
        n = self.n
        R = self.R
        T = self.T
        beta = self.beta
        gamma = self.gamma
        N0 = 10**self.logN0
        alpha = np.log(2) / self.R # slender replication rate
        mu = np.log(2) / T # stumpy death rate

        # populations are stored in lists for plotting
        # committed slenders use a deque list of lists for fast access
        # start with N0 uncommitted slenders and nothing for other cell types
        uncommitted_slenders = [N0]
        committed_slenders = [deque([0]*(n+1))]
        stumpies = [0]
        diff_signal = [0]
        times = [0]

        steps = range(0, int(600/R))

        for s in steps[1:]:
            # last timestep's values
            u_slenders = uncommitted_slenders[-1]
            sum_c_slenders = sum(committed_slenders[-1])
            s = stumpies[-1]
            f = diff_signal[-1]

            # in one timestep (one cell cycle) slenders double of which a proportion p remain uncommitted
            # the rest (proportion 1-p) become committed slenders
            p = np.exp(-beta*f*R)
            uncommitted_slenders.append(2.*u_slenders*p)

            # copy last time step's list of committed slenders into this time step for modification
            committed_slenders.append(deepcopy(committed_slenders[-1]))
            # shortcut for this timestep's committed slenders
            lc = committed_slenders[-1]
            # add new committed slenders from slenders that have just differentiated
            lc.appendleft(2.*u_slenders*(1-p))
            # replicate committeds that became committed in earlier cycles
            for i in range(1, n+1): # number of cell cycles
                lc[i] *= 2

            # add committed slenders that have just become stumpies and remove dead stumpies
            stumpies.append(s*np.exp(-mu*R) + lc.pop())

            # differentiation signal is slenders with natural decay
            z = u_slenders / gamma
            diff_signal.append(z + (f-z)*np.exp(-gamma*R))

        # remove first 96 hours
        xs = int(96 / R)

        # variables to output
        time = np.array(R*np.array(steps[xs:]))
        uncommitted_slenders = np.array(uncommitted_slenders[xs:])
        committed_slenders = np.array([sum(i) for i in committed_slenders[xs:]])
        slenders = uncommitted_slenders + committed_slenders
        stumpies = np.array(stumpies[xs:])
        diff_signal = np.array(diff_signal[xs:])
        total = slenders + stumpies

        return {'time':time, 
                'uncommitted slenders':uncommitted_slenders,
                'committed slenders':committed_slenders,
                'slenders':slenders,
                'stumpies':stumpies,
                'total':total,
                '%proliferative':100 * uncommitted_slenders / total,
                '%slenders':100 * slenders / total,
                'diff_signal':diff_signal}

    ########################################################################################################
    def plot(self, doc):
        source = ColumnDataSource(data=self.simulation())

        total = figure(x_axis_label=r'$$\text{Time post infection (h)}$$', y_axis_label=r'Parasites $$\left(\text{ml}^{-1}\right)$$', y_axis_type='log', y_range=(1e6, 1e10))
        total.line('time', 'total', legend_label='Total', color='DarkBlue', width=3, source=source)
        total.xgrid.visible = False

        slenders = figure(x_axis_label=r'$$\text{Time post infection (h)}$$', y_axis_label=r'$$\text{\% Cells}$$', y_range=(0, 100))
        slenders.line('time', '%slenders', legend_label='Slender', color='Red', width=3, source=source)
        slenders.line('time', '%proliferative', legend_label='Replication competent', color='Green', width=3, dash='dashed', source=source)
        slenders.xgrid.visible = False

        # TODO: y-axis tick labels
        subpops = figure(x_axis_label=r'$$\text{Time post infection (h)}$$', y_axis_label=r'Parasites $$\left(10^{9}\,\text{ml}^{-1}\right)$$', y_range=(0, 1e9))
        subpops.line('time', 'stumpies', legend_label='Stumpies', color='Orange', width=3, source=source)
        subpops.line('time', 'committed slenders', legend_label='Committed slenders', color='Green', width=3, source=source)
        subpops.line('time', 'uncommitted slenders', legend_label='Uncommitted slenders', color='DarkBlue', width=3, source=source)
        subpops.xgrid.visible = False

        ######################### PARAMETER SLIDERS ###########################
        def update(attr, old, new):
            self.n = n.value
            self.R = R.value
            self.T = T.value
            self.beta = beta.value
            self.gamma = gamma.value
            self.logN0 = logN0.value
            source.data = self.simulation()

        f1 = BasicTickFormatter(precision=1)
        n     = Slider(start=0, end=3, value=self.n, step=1, title='no. of divisions of committed slender')
        R     = Slider(start=1, end=10, value=self.R, step=0.1, format='0.0', title='replication time of slenders (h)')
        T     = Slider(start=1, end=40, value=self.T, step=0.1, format='0.0', title='mean stumpy lifespan (h)')
        beta  = Slider(start=1e-10, end=1e-8, value=self.beta, step=1e-10, format=f1, title='sensitivity to differentiation signal')
        gamma = Slider(start=0.001, end=0.1, value=self.gamma, step=0.001, format='0.000', title='differentiation signal decay rate')
        logN0 = Slider(start=0, end=2, value=self.logN0, step=0.1, format='0.0', title='log initial parasite density')
        
        sliders = R, n, T, beta, gamma, logN0 
        
        for w in sliders:
            w.on_change('value', update)

        ######################### RESET BUTTON ###########################
        def reset():
            self.__init__()
            n.value = self.n
            R.value = self.R
            T.value = self.T
            beta.value = self.beta
            gamma.value = self.gamma
            logN0.value = self.logN0
            source.data = self.simulation()
            
        # to reset values, reset has to be called for each slider - don't know why
        button = Button(label="Reset values", button_type="success")
        button.on_event("button_click", *[reset]*len(sliders))
        
        ######################### MODEL NETWORK ###########################

        # list the nodes and initialize a plot
        N = 8
        node_indices = list(range(N))

        model = figure(title="Graph layout demonstration", x_range=(-1.1,1.1),
                      y_range=(-1.1,1.1), tools="", toolbar_location=None)

        graph = GraphRenderer()

        # replace the node glyph with an ellipse
        # set its height, width, and fill_color
        graph.node_renderer.glyph = Ellipse(height=0.1, width=0.2,
                                            fill_color="fill_color")

        # assign a palette to ``fill_color`` and add it to the data source
        graph.node_renderer.data_source.data = dict(
            index=node_indices,
            fill_color=Spectral8)

        # add the rest of the assigned values to the data source
        graph.edge_renderer.data_source.data = dict(
            start=[0]*N,
            end=node_indices)

        # generate ellipses based on the ``node_indices`` list
        circ = [i*2*math.pi/8 for i in node_indices]

        # create lists of x- and y-coordinates
        x = [math.cos(i) for i in circ]
        y = [math.sin(i) for i in circ]

        # convert the ``x`` and ``y`` lists into a dictionary of 2D-coordinates
        # and assign each entry to a node on the ``node_indices`` list
        graph_layout = dict(zip(node_indices, zip(x, y)))

        # use the provider model to supply coourdinates to the graph
        graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

        # render the graph
        model.renderers.append(graph)

        ######################### LAYOUT ###########################
        inputs1 = column(button, *sliders)
        doc.add_root(gridplot(
            children=[[inputs1, None, model], [total, slenders, subpops]],
            width=400, height=400,
            toolbar_location=None
        ))
        


In [3]:
model = model1()
show(model.plot, notebook_url="http://localhost:8888")

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'vscode-webview://1hhc4i1a9ff72vs4cmr3jf68vkh6hhl4e5l7btk9tp55u7q2lr0v';                       use --allow-websocket-origin=1hhc4i1a9ff72vs4cmr3jf68vkh6hhl4e5l7btk9tp55u7q2lr0v or set BOKEH_ALLOW_WS_ORIGIN=1hhc4i1a9ff72vs4cmr3jf68vkh6hhl4e5l7btk9tp55u7q2lr0v to permit this; currently we allow origins {'localhost:8888'}
